In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import openpyxl
from datetime import datetime

In [2]:
def melt(df, col_vals, key, value):
    # melt pandas dataframe
    # col_vars: list of the columns that will be melted
    # key: name of column needs to be generated
    # value: name of the column that contains the value of interest
    keep_vars = df.columns.difference(col_vals)
    melted = []
    for c in col_vals:
        melted_c = df[keep_vars].copy()
        melted_c[key] = c
        melted_c[value] = df[c]
        melted.append(melted_c)
    return pd.concat(melted)

In [1]:
# read the raw spreadsheet, downloanded from shared drive
#c_recon = pd.ExcelFile("C:/celsius/Liquidity/freeze.xlsx")
c_recon = pd.ExcelFile("excel_input/freeze.xlsx")

stats = pd.read_excel(c_recon, "Coin Stats", header = None)
defi = pd.read_excel(c_recon, "DeFi Assets")
defi_main = pd.read_excel(c_recon, "DeFi Main")
ftx = pd.read_excel(c_recon, "FTX Summary")
#template = pd.read_excel("C:/celsius/Liquidity/coin_apy_template.xlsx", sheet_name = "APY", header = None)
template = pd.read_excel("excel_input/coin_apy_template.xlsx", sheet_name = "APY", header = None)
template.columns = template.iloc[0]
template = template[template["Coin"].notnull()]
template

NameError: name 'pd' is not defined

In [4]:
stats[[0,2,3]].head()

,0,2,3
0,Coin/Asset,Net Assets Total,Net Liabilities Total
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,23637082782,-22989724150
4,1INCH,6900893.207,-843737.7119


In [5]:
'''This is computing the ETHER collateral ratio for Aave and Compounds Celsius Borrows Account'''
borrow_account = defi_main[defi_main['Vault Name'] == 'Celsius Borrows Account']
col_ratio = {}
col_threshold = 2.0
protocols = ['Compound', 'Aave V2']

grouped = borrow_account[borrow_account['Coin']=='ETH'].groupby('Protocol')['sum Original Balance'].sum()
grouped = grouped/grouped.sum()

for protocol in protocols:
    temp = borrow_account[borrow_account['Protocol'] == protocol]
    borrow = sum([bal for bal in temp['sum Balance $USD'] if bal<0])
    ether = temp[temp['Coin']=='ETH']['sum Balance $USD'].values[0]
    col_ratio[protocol] = max(abs(ether/borrow) - col_threshold,0) 
    col_ratio[protocol] /= abs(ether/borrow)
    col_ratio[protocol + '_part'] = grouped[protocol]

In [6]:
coin_asset_liability = stats[[0,2,3]].copy()
# Coin/Asset, Net Assets Total, Net Liabilities Total
coin_asset_liability.columns = coin_asset_liability.iloc[0]
coin_asset_liability.drop([0], inplace = True)
coin_asset_liability.dropna(axis = 0 , how = "all", inplace = True)
coin_asset_liability.reset_index(drop = True, inplace = True)
coin_asset_liability.at[0, "Coin/Asset"] = "Total"
coin_asset_liability = coin_asset_liability[coin_asset_liability["Coin/Asset"].notnull()]
coin_asset_liability.columns = ['Coin', 'Net Assets Total', 'Net Liabilities Total']
coin_asset_liability["Net Assets Total"] = coin_asset_liability["Net Assets Total"].astype("float")
coin_asset_liability["Net Liabilities Total"] = coin_asset_liability["Net Liabilities Total"].astype("float")
#coin_asset_liability.columns
coin_asset_liability

,Coin,Net Assets Total,Net Liabilities Total
0,Total,2.363708e+10,-2.298972e+10
1,1INCH,6.900893e+06,-8.437377e+05
2,3CRV,4.072593e+07,-2.026759e+07
3,AAVE,2.191715e+05,-1.961762e+05
4,ADA,2.817839e+08,-2.769298e+08
...,...,...,...
94,YFL,1.518030e+03,0.000000e+00
95,yveCRV-DAO,2.000724e+04,0.000000e+00
96,ZEC,1.214199e+05,-1.218357e+05
97,ZRX,1.447669e+07,-1.456397e+07


In [7]:
# get the coin price, adding stable coins and srm_locked
coin_price = stats[[0,1]]
coin_price.columns = ["Coin", "Price"]
coin_price.drop([0], inplace = True)
coin_price.dropna(axis = 0 , how = "all", inplace = True)
coin_price = coin_price[coin_price["Coin"].notnull()]
coin_price.reset_index(drop = True, inplace = True)
coin_price.loc[len(coin_price.index)] = ['Stable Coins', 1] 
srm_price = float(coin_price.loc[coin_price["Coin"] == "SRM", "Price"])
coin_price.loc[len(coin_price.index)] = ['SRM_LOCKED', srm_price] 
coin_price


/Users/gabrielchoukroun/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/gabrielchoukroun/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Coin,Price
0,1INCH,2.508015
1,3CRV,1.02
2,AAVE,255.924942
3,ADA,1.326447
4,ALCX,313.99
...,...,...
95,ZEC,149.20669
96,ZRX,0.90581
97,ZUSD,0.997757
98,Stable Coins,1


In [8]:
tiers = template[template["Coin"]=="Tier"].T
tiers.reset_index(inplace = True)
tiers.columns = ["Account", "Tier"]
tiers.drop([0], inplace = True)
# there may be duplicate records in tier info
tiers.drop_duplicates(inplace = True)
# there are some rare occasions that the same account is assinged to two or more different tiers, use the first encountered
# the others are likely for some testing purposes, which will usually added to the end 
tiers = tiers[~tiers.Account.duplicated()]
filter4 = tiers['Tier'].isnull()
tiers.at[filter4, "Tier"] = "unassigned"
tiers["Tier"] = tiers["Tier"].astype("string")
#print(len(tiers))
#print(len(tiers[tiers["Tier"] == "unassigned"]))
#print(tiers.Account.nunique())
tiers["Tier"] = tiers["Tier"].apply(lambda x:x.split(".")[0])
#tiers["Tier"].unique()
tiers

,Account,Tier
1,Bank - Balances,1
2,Celsius Network Limited (UK),1
3,Celsius Network LLC (US),1
4,Celsius Network Finance,1
5,Celsius OTC,1
...,...,...
157,TEST-Enzyme Mainnet,2
158,DD - Banker Joe,2
159,DD-Aave-Avalanche,2
160,Yield Desk - Bancor II,2


In [9]:
categories = template[template["Coin"].isin(["Coin", "Category"])].T
categories.reset_index(drop = True, inplace = True)
categories.columns = ["Account", "Category"]
categories.drop([0], inplace = True)
categories.Category.fillna(value = "unassigned", inplace = True)
#print(len(categories))
#print(len(categories[categories["Category"] == "unassigned"]))
#print(categories.Account.nunique())
categories

,Account,Category
1,Bank - Balances,undeployed
2,Celsius Network Limited (UK),undeployed
3,Celsius Network LLC (US),undeployed
4,Celsius Network Finance,undeployed
5,Celsius OTC,undeployed
...,...,...
157,TEST-Enzyme Mainnet,defi
158,DD - Banker Joe,defi
159,DD-Aave-Avalanche,defi
160,Yield Desk - Bancor II,defi


In [10]:
cofa_original = pd.read_excel("excel_input/coin_apy_template.xlsx", sheet_name = "COFA")
cofa_original.dropna(axis=0, how='all', inplace=True)
cofa_original.reset_index(drop = True, inplace = True)
#display(cofa_original)
cofa_original.at[cofa_original["Coin"] == "stable", "Coin"] = "Stable Coins"
cofa_melt_cols = list(cofa_original.columns)
cofa_melt_cols.remove("Coin")
#print(cofa_melt_cols)
cofa = melt(cofa_original, cofa_melt_cols, "Account", "COFA")
cofa = cofa[cofa["COFA"].notnull()]
cofa.reset_index(drop = True, inplace = True)
cofa


,Coin,Account,COFA
0,1INCH,Default,0.0486
1,AAVE,Default,0.0386
2,ADA,Default,0.0234
3,BAT,Default,0.0096
4,BCH,Default,0.0292
...,...,...,...
320,ZEC,Maker Borrows Vault,0.0000
321,ZRX,Maker Borrows Vault,0.0000
322,ZUSD,Maker Borrows Vault,0.0000
323,stable (USD),Maker Borrows Vault,0.0000


In [11]:
apy = template[~template["Coin"].isin(["Coin", "Category", "Tier"])]
#apy.fillna(value = 0, inplace = True)
apy.replace([" ", "", "  "], np.nan, inplace = True)
apy =apy[apy["Coin"] != "wBTC (Y/N)"]
apy

/Users/gabrielchoukroun/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,Coin,Bank - Balances,Celsius Network Limited (UK),Celsius Network LLC (US),Celsius Network Finance,Celsius OTC,CEL Treasury,CEL Users,Loans Out,Posted Collateral,...,DD - SYNAPSIS - USD/Polygon,DD-BENQI-AVALANCHE,DD-ANCHOR-TERRA,DD-SNX,Defi Benqi Deployment,TEST-Enzyme Mainnet,DD - Banker Joe,DD-Aave-Avalanche,Yield Desk - Bancor II,Badger Vote
3,Default,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1INCH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3CRV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AAVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0659,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012,NaN,NaN
8,ADA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.068,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,YFL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,ZEC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0369,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,ZRX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0557,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,ZUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
apy2 = apy.copy()
apy2["Coin"] = apy2["Coin"] + "_APY"
apy5 = apy2.T
apy5.reset_index(inplace = True)
apy5.columns = apy5.iloc[0]
apy5.rename(columns = {"Coin": "Account"}, inplace = True)
apy5.drop([0], inplace = True)
apy5

,Account,Default_APY,1INCH_APY,3CRV_APY,AAVE_APY,ADA_APY,ALCX_APY,ALICE_APY,ALPHA_APY,alUSD_APY,...,XAUT_APY,XLM_APY,XRP_APY,XTZ_APY,YFI_APY,YFL_APY,ZEC_APY,ZRX_APY,ZUSD_APY,check_APY
1,Bank - Balances,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Celsius Network Limited (UK),0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,Celsius Network LLC (US),0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,Celsius Network Finance,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,Celsius OTC,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,TEST-Enzyme Mainnet,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
158,DD - Banker Joe,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
159,DD-Aave-Avalanche,0.0,NaN,NaN,0.012,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
160,Yield Desk - Bancor II,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [13]:
# conver dyptes to float
for col in apy5.columns:
    if col != "Account":
        apy5.loc[apy5[col].isin(template["Coin"].unique()), col] = np.nan
        apy5[col] = apy5[col].astype("float")

In [14]:
# get the user collateral and inst collateral data
collateral = stats.copy()
collateral_p1 = collateral.iloc[:, 0].to_frame()
collateral_p1.columns = ["Coin"]
collateral_p2 = collateral.iloc[:, 4:]
collateral_p2.columns = collateral_p2.iloc[2]
collateral = pd.concat([collateral_p1, collateral_p2[["User Collateral", "Inst Collateral"]]], axis = 1)
collateral.drop([0,1,2,3], inplace = True)
collateral = collateral[collateral["Coin"].notnull()]
collateral["User Collateral"] = -1 * collateral["User Collateral"]
collateral["Inst Collateral"] = -1 * collateral["Inst Collateral"]
collateral.fillna(value = 0, inplace = True)
collateral

,Coin,User Collateral,Inst Collateral
4,1INCH,2.306146e+04,0.00
5,3CRV,0.000000e+00,0.00
6,AAVE,2.664396e+04,12877.61
7,ADA,1.035218e+08,10064338.00
8,ALCX,0.000000e+00,0.00
...,...,...,...
97,YFL,0.000000e+00,0.00
98,yveCRV-DAO,0.000000e+00,0.00
99,ZEC,3.039875e+04,0.00
100,ZRX,7.110053e+05,0.00


In [15]:
# process stats table

# first fill the "asset" or "liability" into row 0
stats.iloc[0] = stats.iloc[0].ffill()
# drop the secondary description of asset or liability (no use)
stats.drop([1], inplace = True)

#split stats into two parts, p1 is coin name and summary, p2 is assets/liabilities
stats_p1 = stats.iloc[:, 0:3]
stats_p1.columns = stats_p1.iloc[0]
stats_p1.reset_index(drop = True, inplace = True)
stats_p1.drop([0, 1, 2], inplace = True)
stats_p1.rename(columns = {"Coin/Asset": "Coin"}, inplace = True)


stats_p2 = stats.iloc[:, 4:]

# filter according to "assets" and the detailed account name cannot be null
stats_p3 = stats_p2.loc[:, stats_p2.loc[2].notnull()]
stats_p4 = stats_p3.loc[:, stats_p3.loc[0] == "Assets"]
stats_p4.reset_index(drop = True, inplace = True)
stats_p4.columns = stats_p4.iloc[1]
stats_p4.reset_index(drop = True, inplace = True)
stats_p4.drop([0, 1, 2], inplace = True)
stats = pd.concat([stats_p1["Coin"], stats_p4], axis = 1)
stats = stats[stats["Coin"].notnull()]
stats

/Users/gabrielchoukroun/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/gabrielchoukroun/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Coin,Bank - Balances,Celsius Network Limited (UK),Celsius Network LLC (US),Celsius Network EU UAB (LT),Celsius OTC,Celsius Network Finance,CEL Treasury,CEL Users,Loans Out,...,Stakehound,Kraken Staking,Direct Staking,Mining,EAM - Balances,Others - Asset,BITFINEX,COINBASEPRO,DERIBIT,LIQUID
3,1INCH,0,0,234226.5466,0,0,0,0,0,0,...,0,0,0,0,0,6666666.66,0,0,0,0
4,3CRV,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,AAVE,0,12877.66549,10031.47961,0,106.934225,5.93778,0,0,21432.7,...,0,0,0,0,0,0,0,0,0,0
6,ADA,0,4781.059044,14412934.85,0,14728.44519,268.036039,0,0,88361559,...,0,0,178989656,0,0,0,0,0,0,0
7,ALCX,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,YFL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,yveCRV-DAO,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,ZEC,0,14.6212,82774.69866,0,0,0,0,0,34000,...,0,0,0,0,0,0,4630.618948,0,0,0
99,ZRX,0,137795.3891,951449.6104,0,0,272.88,0,0,3500000,...,0,0,0,0,0,0,577453.4256,8735.95141,0,0


In [16]:
defi_p1 = defi.iloc[:, 0:3]
defi_p1.drop([0, 1, 2], inplace = True)
defi_p1.rename(columns = {"Coin/Asset": "Coin"}, inplace = True)
defi_p1
defi_p2 = defi.iloc[:, 3:]

# filter according the detailed account name cannot be null
defi_p3 = defi_p2.loc[:, defi_p2.iloc[1].notnull()]
defi_p3.reset_index(drop = True, inplace = True)
defi_p3.columns = defi_p3.iloc[1]
defi_p3.reset_index(drop = True, inplace = True)
defi_p3.drop([0, 1, 2], inplace = True)
defi_p3
defi = pd.concat([defi_p1["Coin"], defi_p3], axis = 1)
defi = defi[defi["Coin"].notnull()]
defi

/Users/gabrielchoukroun/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Coin,Deployment - Curve,OmniMan1,YD - Convex - LUSD,YD - Curve - BUSDv2,YieldDesk_Main,DD - FRAX - Convex,DD-CONVEX-ALUSD,DD-CONVEX-EURT,DD-CONVEX-OUSD,...,YD - BADGER - byvWBTC,YD - Badger - SBTC,YD - Badger - renBTC,YD - Curve - SLINK,YD - Curve - pBTC,YD - Vesper - vETH,YD - Vesper - vLINK,YD - Vesper - vWBTC,YFL,Yield Desk - Compound
3,1INCH,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3CRV,0,1975.207784,13954802.94,2327297.431,2328.773411,14782787.4,8759390.553,474869.4899,422482.5531,...,0,0,0,0,0,0,0,0,0,0
5,AAVE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,ADA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,ALCX,0,0,0,0,0,0,41.568835,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,YFL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1518.03038,0
97,yveCRV-DAO,0,10000,0,0,10007.23888,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,ZEC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,ZRX,0,0.001928,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# ADD GAB - SPLIT ON OVERCOLLATERIZED ETH IN CELSIUS BORROWS ACCOUNT
borrow_eth = defi[defi['Coin']=='ETH']['Celsius Borrows Account'].iloc[0]
idx_eth = defi[defi['Coin']=='ETH'].index[0]
for col in ['Aave V2', 'Compound']:
    new_col = f'Overcollaterized - {col}'
    defi[new_col] = 0
    defi.at[idx_eth,new_col] = borrow_eth * col_ratio[col+'_part']* col_ratio[col]
    defi.at[idx_eth,'Celsius Borrows Account'] -= defi.at[idx_eth,new_col]

In [18]:
# process ftx summary data
ftx.dropna(axis = 0 , how = "all", inplace = True)
ftx.dropna(axis = 1 , how = "all", inplace = True)
filter_ftx = ((ftx["Total Asset"] == 0) & (ftx["Total Borrow"] == 0))
ftx = ftx[(ftx["coin"].notnull()) & ~filter_ftx]
ftx.drop(columns = ["Total Asset", "Total Borrow"], inplace = True)
ftx_cols = []
for col in ftx.columns:
    ftx_cols.append("FTX - " + col)
ftx.columns = ftx_cols
ftx.rename(columns = {"FTX - coin": "Coin"}, inplace = True)

# below lines of code will merge BTC and WBTC together
ftx_btc = ftx[ftx["Coin"].isin(["BTC", "WBTC"])]
ftx_btc = ftx_btc.append(ftx_btc.sum(numeric_only=True), ignore_index=True)
ftx_btc = ftx_btc[~ftx_btc["Coin"].isin(["BTC", "WBTC"])]
ftx_btc.reset_index(drop = True, inplace = True)
ftx_btc.loc[0, "Coin"] = "BTC"
ftx = pd.concat([ftx_btc, ftx[~ftx["Coin"].isin(["BTC", "WBTC"])]])
ftx

,Coin,FTX - CnC,FTX - TEAM Directional,FTX - Grayscale,FTX - Main Account,FTX - DeFi,FTX - Management,FTX - Kairon,FTX - Kairon2,FTX - Brad,FTX - Jacob,FTX - cel_staking,FTX - Directional Trading 2,FTX - CEL,FTX - Borrow
0,BTC,3.776284e+03,0.000000e+00,3.874041e+01,0.000000e+00,13.000390,0.000000e+00,4.949090,0.000000e+00,0.000000e+00,20509.931128,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
0,USD,2.199383e+07,-1.249023e+06,1.622286e+07,1.007122e+06,491253.299619,2.050419e+06,253944.978528,9.067869e+01,6.022710e+06,0.000000,0.000000e+00,1.988425e+06,3.647564e+06,17.115227
1,SRM_LOCKED,6.345008e+05,1.061677e+05,0.000000e+00,8.185316e+04,0.000000,7.481214e+03,0.000000,0.000000e+00,1.343418e+04,0.000000,4.039846e+05,0.000000e+00,2.547981e+03,0.000000
2,SRM,5.619351e+03,3.032213e+03,0.000000e+00,2.001856e+07,0.000000,9.408591e+01,0.000000,0.000000e+00,2.458210e+02,0.000000,1.605368e+08,0.000000e+00,1.701930e+01,0.000000
3,AAVE,1.500039e+04,0.000000e+00,0.000000e+00,3.756150e-03,0.061633,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
4,FTT,4.740586e+06,1.000000e+06,0.000000e+00,1.000000e+01,0.000000,3.562740e+04,0.000000,0.000000e+00,2.000000e+05,0.000000,7.120059e+06,0.000000e+00,1.250000e+05,0.000000
5,LTC,9.606779e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,110147.195792,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
6,ETH,1.445787e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,202861.439456,0.000000e+00,0.000000e+00,0.000000e+00,0.100000
7,ZRX,6.062600e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
11,BADGER,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.002652,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000


In [19]:
# get update date and time
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")
#print("Updated at:", dt_string)

In [20]:
# define the stable coins
stables = ['alUSD','BUSD', 'GUSD', 'LUSD', 'LUSD Curve','MCDAI', 'PAX', 'SUSD', 'TUSD', 
           'USDC', 'USDT ERC20', 'ZUSD', "USD"]


In [21]:
# get a pandas df containing all tiers for merging
tier_dict = {"Tier":["1", "2", "3", "4", "5", "unassigned"]}
coin_tiers = pd.DataFrame.from_dict(tier_dict)
coin_tiers

,Tier
0,1
1,2
2,3
3,4
4,5
5,unassigned


In [22]:
# merge all 3 parts (coin stats, defi assets and ftx summary) together 
coin_stats = stats.merge(defi, on = "Coin", how = "outer")
coin_stats = coin_stats.merge(ftx, on = "Coin", how = "outer")
coin_stats.fillna(value = 0, inplace = True)
coin_stats_col = []
# the follwoing code is to remove the spaces trailing or following the name of the columns
for col in coin_stats.columns:
    coin_stats_col.append(col.strip())
coin_stats.columns = coin_stats_col
#len(coin_stats.columns)
coin_stats

,Coin,Bank - Balances,Celsius Network Limited (UK),Celsius Network LLC (US),Celsius Network EU UAB (LT),Celsius OTC,Celsius Network Finance,CEL Treasury,CEL Users,Loans Out,...,FTX - DeFi,FTX - Management,FTX - Kairon,FTX - Kairon2,FTX - Brad,FTX - Jacob,FTX - cel_staking,FTX - Directional Trading 2,FTX - CEL,FTX - Borrow
0,1INCH,0.0,0.000000,2.342265e+05,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0
1,3CRV,0.0,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0
2,AAVE,0.0,12877.665490,1.003148e+04,0.0,106.934225,5.937780,0.0,0.0,21432.7,...,0.061633,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0
3,ADA,0.0,4781.059044,1.441293e+07,0.0,14728.445190,268.036039,0.0,0.0,88361559.0,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0
4,ALCX,0.0,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,ZUSD,0.0,0.000000,7.490650e+05,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0
98,SRM_LOCKED,0.0,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,7481.214094,0.0,0.0,13434.17896,0.0,403984.628169,0.0,2547.980701,0.0
99,ALICE,0.0,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0
100,LRC,0.0,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0


In [23]:
# get a list of coins 
coin_list = list(coin_stats["Coin"].unique())
coin_list.append("Stable Coins")
#coin_list

In [24]:
# get a list of accounts
acc_list = np.asarray(tiers["Account"])
len(acc_list)

161

In [25]:
# see which accounts in coin recon sheet are not in covered in apy/cofa sheet
not_covered = []
for col in coin_stats.columns:
    if col not in acc_list:
        not_covered.append(col)
not_covered    

['Coin',
 'Celsius Network EU UAB (LT)',
 'DeFi Borrows - Collateral',
 'DeFi Borrows - Tokens',
 'DeFi Assets - Assets',
 'FTX',
 'DD-CONVEX-EURT',
 'DD-CONVEX-OUSD',
 'YD - AlphaHv2- USDC.e',
 'DD-CONVEX-3EURpool',
 'DD - Benqi - Avalanche',
 'Test Avalanche 2',
 'DD-CONVEX-EURS',
 'DD-CONVEX-d3pool',
 'DD-Anchor-Terra',
 'AAVE ARC Borrows',
 'AAVE ARC Supply',
 'DD - SNX',
 'Deployment - Vesper (OLD DO NOT USE)',
 'SGB Staking']

In [26]:
# see which accounts in apy/cofa sheet are not listed in coin recon sheet
not_covered = []
for col in acc_list:
    if col not in coin_stats.columns:
        not_covered.append(col)
not_covered 

['Impermanent_loss_hedge__RonSabo',
 'Hedge_Options',
 'FTX - Johannes',
 'OKEX',
 'YD-AlphaHv2-USDC.e',
 'YD - Curve - renBTC',
 'Deployment - Vesper',
 'YD - Compound - WBTC',
 'BlockDemon ETH Staking',
 'Deployment - Stable Coin Swaps',
 'Deployment Team - COMP supply',
 'Deployment- 1INCH Staking',
 'Convex: cvxBUSD3CRV-f',
 'FTX - LONG1',
 0.0,
 'YD - Curve - SBTC',
 'YD - Keeper - renBTC',
 'MATIC Staking 7',
 'Deployment - 1INCH Staking (testing)',
 'TEST AVALANCHE 2',
 'YD - Curve - alUSD',
 'DD-CONVEX-3EURpool (ethereum network)',
 'DD-CONVEX-EURT (ethereum network)',
 'DD-CONVEX-EURS (ethereum network)',
 'DD-CONVEX-OUSD (ethereum network)',
 'DD-CONVEX-d3pool (ethereum network)',
 'DD - SYNAPSIS - WETH/Arbitrum',
 'DD - SYNAPSIS - WETH/Optimism',
 'DD - SYNAPSIS - USD/Avalanche',
 'DD - SYNAPSIS - USD/Polygon',
 'DD-BENQI-AVALANCHE',
 'DD-ANCHOR-TERRA',
 'DD-SNX',
 'DD - Banker Joe',
 'DD-Aave-Avalanche',
 'Yield Desk - Bancor II']

In [27]:
# transpose coin_stats so we can merge with Tier/APY/COFA on accounts
coin_stats_t = coin_stats.T
new_header = coin_stats_t.iloc[0] #grab the first row for the header
coin_stats_t = coin_stats_t[1:] #take the data less the header row
coin_stats_t.columns = new_header #set the header row as the df header
coin_stats_t.reset_index(inplace = True)
coin_stats_t.rename(columns={coin_stats_t.columns[0]: "Account" }, inplace = True)
coin_stats_t.reset_index(drop=True, inplace = True)
coin_stats_t = coin_stats_t[~coin_stats_t["Account"].isin(['DeFi Borrows - Assets', 'DeFi Assets - Assets', "FTX"])]

coin_stats_t


Coin,Account,1INCH,3CRV,AAVE,ADA,ALCX,ALPHA,alUSD,AMPL,ANKR,...,YFI,YFL,yveCRV-DAO,ZEC,ZRX,ZUSD,SRM_LOCKED,ALICE,LRC,TRX
0,Bank - Balances,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Celsius Network Limited (UK),0.0,0.0,12877.66549,4781.059044,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,14.6212,137795.3891,0.0,0.0,0.0,0.0,0.0
2,Celsius Network LLC (US),234226.5466,0.0,10031.47961,14412934.85,0.0,0.0,0.0,203.249711,24004.58859,...,0.308855,0.0,0.0,82774.69866,951449.6104,749064.9677,0.0,0.0,0.0,0.0
3,Celsius Network EU UAB (LT),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Celsius OTC,0.0,0.0,106.934225,14728.44519,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,FTX - Jacob,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140,FTX - cel_staking,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,403984.628169,0.0,0.0,0.0
141,FTX - Directional Trading 2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
142,FTX - CEL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2547.980701,0.0,0.0,0.0


In [28]:
# merge the tier info 
coin_stat_tier = tiers.merge(coin_stats_t, on = "Account", how = "right")
#coin_stat_tier["Tier"].fillna(value = "unknown", inplace = True)

# get the number of stable coins
coin_stat_tier['Stable Coins']= coin_stat_tier[stables].sum(axis=1)
coin_stat_tier.to_excel("coin_stats_tier.xlsx", index = False)

#print(len(coin_stat_tier[coin_stat_tier["Tier"] == "unknown"]))
coin_stat_tier["Tier"] = coin_stat_tier["Tier"].astype("str")
coin_stat_tier

,Account,Tier,1INCH,3CRV,AAVE,ADA,ALCX,ALPHA,alUSD,AMPL,...,YFL,yveCRV-DAO,ZEC,ZRX,ZUSD,SRM_LOCKED,ALICE,LRC,TRX,Stable Coins
0,Bank - Balances,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.901267e+07
1,Celsius Network Limited (UK),1,0.0,0.0,12877.66549,4781.059044,0.0,0.0,0.0,0.0,...,0.0,0.0,14.6212,137795.3891,0.0,0.0,0.0,0.0,0.0,6.182790e+06
2,Celsius Network LLC (US),1,234226.5466,0.0,10031.47961,14412934.85,0.0,0.0,0.0,203.249711,...,0.0,0.0,82774.69866,951449.6104,749064.9677,0.0,0.0,0.0,0.0,4.557011e+07
3,Celsius Network EU UAB (LT),nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
4,Celsius OTC,1,0.0,0.0,106.934225,14728.44519,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.516606e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,FTX - Jacob,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
138,FTX - cel_staking,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,403984.628169,0.0,0.0,0.0,0.000000e+00
139,FTX - Directional Trading 2,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.988425e+06
140,FTX - CEL,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2547.980701,0.0,0.0,0.0,3.647564e+06


In [29]:
# creating a list for summing stable coin yields
stable_yield = []
for coin in stables:
    stable_yield.append(coin+"_yield")
stable_yield

['alUSD_yield',
 'BUSD_yield',
 'GUSD_yield',
 'LUSD_yield',
 'LUSD Curve_yield',
 'MCDAI_yield',
 'PAX_yield',
 'SUSD_yield',
 'TUSD_yield',
 'USDC_yield',
 'USDT ERC20_yield',
 'ZUSD_yield',
 'USD_yield']

In [30]:
coin_stat_tier_apy = coin_stat_tier.merge(apy5, on = "Account", how = "left")
coin_stat_tier_apy.head(10)

,Account,Tier,1INCH,3CRV,AAVE,ADA,ALCX,ALPHA,alUSD,AMPL,...,XAUT_APY,XLM_APY,XRP_APY,XTZ_APY,YFI_APY,YFL_APY,ZEC_APY,ZRX_APY,ZUSD_APY,check_APY
0,Bank - Balances,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,Celsius Network Limited (UK),1,0.0,0.0,12877.66549,4781.059044,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,Celsius Network LLC (US),1,234226.5466,0.0,10031.47961,14412934.85,0.0,0.0,0.0,203.249711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,Celsius Network EU UAB (LT),nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Celsius OTC,1,0.0,0.0,106.934225,14728.44519,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,Celsius Network Finance,1,0.0,0.0,5.93778,268.036039,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
6,CEL Treasury,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
7,CEL Users,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
8,Loans Out,4,0.0,0.0,21432.7,88361559.0,0.0,0.0,0.0,0.0,...,NaN,0.0503,0.0462,0.0625,0.0831,NaN,0.0369,0.0557,NaN,44.0
9,Posted Collateral,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [31]:
#coin_stat_tier_apy.columns.tolist() 
# Default_APY columns

In [32]:
# calculating the yield for each coin/account
coin_stat_tier_apy = coin_stat_tier.merge(apy5, on = "Account", how = "left")

In [33]:
# ADD --------------------------
# ------------------------------
# I need to add a function that will fill _APY columns with the default values
def default_fill(row):
    if ~ np.isnan(row['Default_APY']): 
        row.loc[apy_cols] = row[apy_cols].fillna(row['Default_APY'])
    return row

In [34]:
# calculating the yield for each coin/account
coin_stat_tier_apy = coin_stat_tier.merge(apy5, on = "Account", how = "left")
#coin_stat_tier_apy.fillna(value = 0, inplace = True)
# --------- ADD GAB - Replace NA APY by the corresponding Default APY values if it exists -------------------
apy_cols = ['_APY' in col for col in coin_stat_tier_apy.columns]
coin_stat_tier_apy = coin_stat_tier_apy.apply(default_fill, axis=1, result_type='broadcast')
coin_stat_tier_apy['Stable Coins'] = coin_stat_tier_apy['Stable Coins'].astype(np.float64)
# ------------------------------------------------------------------------------
#coin_stat_tier_apy.to_excel("C:/celsius/Liquidity/coin_stat_tier_apy.xlsx", index = False)
for coin in coin_list:
    #print(coin)
    yield_name = coin + "_yield"
    coin_apy = coin + "_APY"
    if coin_apy in coin_stat_tier_apy.columns:
        coin_stat_tier_apy[yield_name] = coin_stat_tier_apy[coin] * coin_stat_tier_apy[coin_apy]
    else:
        coin_stat_tier_apy[yield_name] = 0
        coin_stat_tier_apy[coin_apy] = np.nan
coin_stat_tier_apy['Stable Coins_yield']= coin_stat_tier_apy[stable_yield].sum(axis=1)
coin_stat_tier_apy['Stable Coins_APY']= coin_stat_tier_apy['Stable Coins_yield']/ coin_stat_tier_apy['Stable Coins']
#coin_stat_tier_apy.to_excel("C:/celsius/Liquidity/coin_stat_tier_apy.xlsx", index = False)
coin_stat_tier_apy.to_excel("excel_outputs/coin_stat_tier_apy.xlsx", index = False)
coin_stat_tier_apy

/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_3411/2289966129.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  coin_stat_tier_apy[yield_name] = coin_stat_tier_apy[coin] * coin_stat_tier_apy[coin_apy]
/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_3411/2289966129.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  coin_stat_tier_apy[yield_name] = 0
/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_3411/2289966129.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually t

,Account,Tier,1INCH,3CRV,AAVE,ADA,ALCX,ALPHA,alUSD,AMPL,...,ZEC_yield,ZRX_yield,ZUSD_yield,SRM_LOCKED_yield,ALICE_yield,LRC_yield,TRX_yield,TRX_APY,Stable Coins_yield,Stable Coins_APY
0,Bank - Balances,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,0.000000e+00,0.000000
1,Celsius Network Limited (UK),1,0.0,0.0,12877.66549,4781.059044,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,0.000000e+00,0.000000
2,Celsius Network LLC (US),1,234226.5466,0.0,10031.47961,14412934.85,0.0,0.0,0.0,203.249711,...,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,0.000000e+00,0.000000
3,Celsius Network EU UAB (LT),nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0.000000e+00,NaN
4,Celsius OTC,1,0.0,0.0,106.934225,14728.44519,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,0.000000e+00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,FTX - Jacob,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0.000000e+00,NaN
138,FTX - cel_staking,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,0.000000e+00,NaN
139,FTX - Directional Trading 2,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0,NaN,-3.437336e+06,-1.728672
140,FTX - CEL,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,0.000000e+00,0.000000


In [35]:
# group by coin/tier
for coin in coin_list:
    #print(coin)
    coin_yield = coin+"_yield"
    coin_apy = coin+"_APY"
    df1 = coin_stat_tier_apy[[coin, coin_yield, "Tier"]]
    df2 = df1.groupby('Tier', as_index = False).agg({coin: "sum",
                                                    coin_yield: "sum"})
    df2[coin_apy] = df2[coin_yield]/ df2[coin]
    df2.drop(columns = [coin_yield], inplace = True)
    #display(df2)
    coin_tiers = coin_tiers.merge(df2, on = "Tier", how = "left")
coin_tiers.fillna(value = 0, inplace = True)
coin_tiers

,Tier,1INCH,1INCH_APY,3CRV,3CRV_APY,AAVE,AAVE_APY,ADA,ADA_APY,ALCX,...,SRM_LOCKED,SRM_LOCKED_APY,ALICE,ALICE_APY,LRC,LRC_APY,TRX,TRX_APY,Stable Coins,Stable Coins_APY
0,1,2.342265e+05,0.0,0.000000e+00,0.000000,23252.056014,0.000000,1.443271e+07,0.000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,1.357342e+08,0.000000
1,2,0.000000e+00,0.0,3.982428e+07,0.071642,0.003756,0.000000,0.000000e+00,0.000,41.568835,...,84401.137379,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,1.080146e+08,0.074089
2,3,0.000000e+00,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000e+00,0.000000
3,4,0.000000e+00,0.0,0.000000e+00,0.000000,36433.148351,-0.071248,8.836156e+07,0.068,0.000000,...,761583.895809,-0.216938,60000.047441,-0.267206,900021.23058,-0.267206,0.000003,0.0,1.363345e+09,0.080987
4,5,6.666667e+06,0.0,4.303981e+03,0.000000,159486.257209,0.064300,0.000000e+00,0.000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,5.333025e+08,0.216845
5,unassigned,0.000000e+00,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000e+00,0.000000


In [36]:
coin_tier_p1_cols = ["Tier"]
coin_tier_p2_cols = ["Tier"]
filter2 = coin_tiers["Tier"] != "unassigned"
for col in coin_tiers.columns:
    if col != "Tier" and "_APY" in col:
        coin_tier_p2_cols.append(col)
    elif col != "Tier":
        coin_tier_p1_cols.append(col)
    else:
        pass
print(coin_tier_p1_cols)
print(coin_tier_p2_cols)

['Tier', '1INCH', '3CRV', 'AAVE', 'ADA', 'ALCX', 'ALPHA', 'alUSD', 'AMPL', 'ANKR', 'ATLAS', 'AVAX', 'BADGER', 'BAL', 'BAT', 'BCH', 'BNB', 'BNT', 'BOBA', 'BOND', 'BOR', 'BSV', 'BTC', 'BTG', 'BUSD', 'CEL', 'COMP', 'CREAM', 'CRV', 'CVX', 'DASH', 'DIGG', 'DOT', 'EOS', 'ETC', 'ETH', 'FARM', 'FIS', 'FTM', 'FTT', 'GUSD', 'KIN', 'KNC', 'LDO', 'LINK', 'LPT', 'LQTY', 'LTC', 'LUNA', 'LUSD', 'LUSD Curve', 'MANA', 'MATIC', 'MCDAI', 'MKR', 'OMG', 'ONX', 'ORBS', 'PAX', 'PAXG', 'PNT', 'POLIS', 'QI', 'RAY', 'REN', 'ROOK', 'SGA', 'SGB', 'SGR', 'SNX', 'SOL', 'SPARK', 'SRM', 'SUSD', 'SUSHI', 'TAUD', 'TCAD', 'TGBP', 'THKD', 'TRU', 'TUSD', 'UMA', 'UNI', 'USD', 'USDC', 'USDT ERC20', 'VSP', 'WBTC', 'WDGLD', 'XAUT', 'XLM', 'XRP', 'XTZ', 'YFI', 'YFL', 'yveCRV-DAO', 'ZEC', 'ZRX', 'ZUSD', 'SRM_LOCKED', 'ALICE', 'LRC', 'TRX', 'Stable Coins']
['Tier', '1INCH_APY', '3CRV_APY', 'AAVE_APY', 'ADA_APY', 'ALCX_APY', 'ALPHA_APY', 'alUSD_APY', 'AMPL_APY', 'ANKR_APY', 'ATLAS_APY', 'AVAX_APY', 'BADGER_APY', 'BAL_APY', 'BAT_A

In [37]:
coin_tier_p1 = coin_tiers[filter2][coin_tier_p1_cols].T
coin_tier_p1.reset_index(inplace = True)
coin_tier_p1.columns = coin_tier_p1.iloc[0]
coin_tier_p1.drop([0], inplace = True)
new_name = ["Coin"]
for col in coin_tier_p1.columns:
    if col != "Tier":
        new_name.append("Coin_Tier_"+str(col)[0])
coin_tier_p1.columns = new_name
coin_tier_p1

,Coin,Coin_Tier_1,Coin_Tier_2,Coin_Tier_3,Coin_Tier_4,Coin_Tier_5
1,1INCH,234226.5466,0.0,0.0,0.0,6666666.66
2,3CRV,0.0,39824278.324,0.0,0.0,4303.981195
3,AAVE,23252.056014,0.003756,0.0,36433.148351,159486.257209
4,ADA,14432712.390273,0.0,0.0,88361559.0,0.0
5,ALCX,0.0,41.568835,0.0,0.0,0.0
...,...,...,...,...,...,...
99,SRM_LOCKED,0.0,84401.137379,0.0,761583.895809,0.0
100,ALICE,0.0,0.0,0.0,60000.047441,0.0
101,LRC,0.0,0.0,0.0,900021.23058,0.0
102,TRX,0.0,0.0,0.0,0.000003,0.0


In [38]:
coin_tier_p2 = coin_tiers[filter2][coin_tier_p2_cols]
coin_tier_p2.columns = coin_tier_p1_cols
coin_tier_p2 = coin_tier_p2.T
coin_tier_p2.reset_index(inplace = True)
coin_tier_p2.columns = coin_tier_p2.iloc[0]
coin_tier_p2.drop([0], inplace = True)
new_name = ["Coin"]
for col in coin_tier_p2.columns:
    if col != "Tier":
        new_name.append("APY_Tier_"+str(col)[0])
coin_tier_p2.columns = new_name
coin_tier_p2

,Coin,APY_Tier_1,APY_Tier_2,APY_Tier_3,APY_Tier_4,APY_Tier_5
1,1INCH,0.0,0.0,0.0,0.0,0.0
2,3CRV,0.0,0.071642,0.0,0.0,0.0
3,AAVE,0.0,0.0,0.0,-0.071248,0.0643
4,ADA,0.0,0.0,0.0,0.068,0.0
5,ALCX,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
99,SRM_LOCKED,0.0,0.0,0.0,-0.216938,0.0
100,ALICE,0.0,0.0,0.0,-0.267206,0.0
101,LRC,0.0,0.0,0.0,-0.267206,0.0
102,TRX,0.0,0.0,0.0,0.0,0.0


In [39]:
coin_tiers = coin_tier_p1.merge(coin_tier_p2, on = "Coin", how = "inner")
filter3 = ~coin_tiers["Coin"].isin(stables)
#coin_tiers.columns
coin_tiers

,Coin,Coin_Tier_1,Coin_Tier_2,Coin_Tier_3,Coin_Tier_4,Coin_Tier_5,APY_Tier_1,APY_Tier_2,APY_Tier_3,APY_Tier_4,APY_Tier_5
0,1INCH,234226.5466,0.0,0.0,0.0,6666666.66,0.0,0.0,0.0,0.0,0.0
1,3CRV,0.0,39824278.324,0.0,0.0,4303.981195,0.0,0.071642,0.0,0.0,0.0
2,AAVE,23252.056014,0.003756,0.0,36433.148351,159486.257209,0.0,0.0,0.0,-0.071248,0.0643
3,ADA,14432712.390273,0.0,0.0,88361559.0,0.0,0.0,0.0,0.0,0.068,0.0
4,ALCX,0.0,41.568835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
98,SRM_LOCKED,0.0,84401.137379,0.0,761583.895809,0.0,0.0,0.0,0.0,-0.216938,0.0
99,ALICE,0.0,0.0,0.0,60000.047441,0.0,0.0,0.0,0.0,-0.267206,0.0
100,LRC,0.0,0.0,0.0,900021.23058,0.0,0.0,0.0,0.0,-0.267206,0.0
101,TRX,0.0,0.0,0.0,0.000003,0.0,0.0,0.0,0.0,0.0,0.0


## the below code is to generat the pivot_data tab

In [40]:
some_dict = {"Coin":[], "Category":[], "Account": [], "Tier": [], "# of Coins": [], "APY": []}
df1 = pd.DataFrame.from_dict(some_dict)
col_names = ["Coin", "Category", "Account", "Tier", "# of Coins", 
             "APY", "COFA", "USD Value", "USD Value * COFA",
             "USD Value * APY"]
df1

,Coin,Category,Account,Tier,# of Coins,APY


In [41]:
for coin in coin_list:
    if coin not in stables:
        used_cols = ["Account", "Tier", coin, coin+"_APY"]
        df2 = categories.merge(coin_stat_tier_apy[used_cols], on = "Account", how = "right")
        df2["Coin"] = coin
        df2.rename(columns = {coin: "# of Coins", coin+"_APY": "APY"}, inplace = True)
        df1 = pd.concat([df1, df2])
filter1 = df1["# of Coins"] > 10e-6
df1 = df1[filter1]
df1 = df1.merge(coin_price, on = "Coin", how = "left")
df1["USD Value"] = df1["# of Coins"] * df1["Price"]
df1["USD Value * APY"] = df1["USD Value"] * df1["APY"]
df1["Tier"] = df1["Tier"].apply(lambda x:x.split(".")[0])
df1

,Coin,Category,Account,Tier,# of Coins,APY,Price,USD Value,USD Value * APY
0,1INCH,undeployed,Celsius Network LLC (US),1,234226.5466,0.0,2.508015,587443.748017,0.0
1,1INCH,Other,Others - Asset,5,6666666.66,0.0,2.508015,16720101.569947,0.0
2,3CRV,Defi,OmniMan1,5,1975.207784,0.0,1.02,2014.71194,0.0
3,3CRV,defi,YD - Convex - LUSD,2,13954802.94,0.0,1.02,14233898.9988,0.0
4,3CRV,Defi,YD - Curve - BUSDv2,2,2327297.431,0.0,1.02,2373843.37962,0.0
...,...,...,...,...,...,...,...,...,...
670,Stable Coins,Operational,FTX - Kairon2,4,90.678693,0.0,1,90.678693,0.0
671,Stable Coins,Exchange,FTX - Brad,4,6118172.083626,0.321985,1,6118172.083626,1969960.404341
672,Stable Coins,Exchange,FTX - Directional Trading 2,4,1988425.11367,-1.728672,1,1988425.11367,-3437335.643294
673,Stable Coins,undeployed,FTX - CEL,2,3647563.643993,0.0,1,3647563.643993,0.0


In [42]:
# below 3 code blocks are for analyzing COFA
cats = list(df1["Category"].unique())
accs = list(df1["Account"].unique())
cofa_accts = list(cofa["Account"].unique())
cofa_accts.remove("Default")
cofa_accts

['CEL Treasury',
 'Posted Collateral',
 'FTX - CnC',
 'FTX - CEL',
 'FTX - TEAM Directional',
 'FTX - LONG1',
 'Celsius Borrows Account',
 'FTX - Borrow',
 'Maker Borrows Vault']

In [43]:
def add_cofa(df1, cofa, cofa_accts):
    default_cofa = cofa[cofa["Account"] == "Default"][["Coin", "COFA"]]
    df2 = df1.merge(default_cofa, on = "Coin", how = "left")
    for col in cofa_accts:
        #print(col)
        cofa_2 = cofa[cofa["Account"] == col]
        if col in cats:
            cofa_2.columns = ["Coin", "Category", "COFA-2"]
            df2 = df2.merge(cofa_2, on=["Coin", "Category"], how='left')
        elif col in accs:
            cofa_2.columns = ["Coin", "Account", "COFA-2"]
            df2 = df2.merge(cofa_2, on=["Coin", "Account"], how='left')
        else:
            continue
        df2_p1 = df2[df2["COFA-2"].isnull()].drop(columns = ["COFA-2"])
        df2_p2 = df2[df2["COFA-2"].notnull()]
        df2_p2["COFA"] = df2_p2["COFA-2"]
        df2_p2.drop(columns = ["COFA-2"], inplace = True)
        df2 = pd.concat([df2_p1, df2_p2])
        df2.sort_values(by = ["Coin"], inplace = True)
    return df2



In [44]:
df2 = add_cofa(df1, cofa, cofa_accts)
#df2.to_excel("C:/celsius/Liquidity/cofa_2.xlsx", index = False)
df2.to_excel("cofa_2.xlsx", index = False)
df2["USD Value * COFA"] = df2["USD Value"] * df2["COFA"]
df2.sort_values(by = "USD Value", ascending = False, inplace = True)
df2['Category'] = df2['Category'].str.strip()
df2

/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_3411/2289702927.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_p2["COFA"] = df2_p2["COFA-2"]
/Users/gabrielchoukroun/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Coin,Category,Account,Tier,# of Coins,APY,Price,USD Value,USD Value * APY,COFA,USD Value * COFA
335,ETH,Posted Collateral,Celsius Borrows Account,5,521449.2824,0.003,3817.111711,1990430162.541586,5971290.487625,0.0000,0.0
332,ETH,NaN,DeFi Borrows - Collateral,nan,520431.498,NaN,3817.111711,1986545165.789073,NaN,0.0376,74694098.233669
133,BTC,Posted Collateral,Posted Collateral,5,40207.62,0.0,46206.66951,1857860209.123666,0.0,0.0000,0.0
121,BTC,Institutional Loans,Loans Out,4,28024.63,0.0294,46206.66951,1294924816.550031,38070789.606571,0.0294,38070789.606571
530,Stable Coins,Institutional Loans,Loans Out,4,1248208900.0,0.086525,1,1248208900.0,108000756.9,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
513,Stable Coins,defi,DD-CURVE-aTricrypto.e,4,0.001409,0.19,1,0.001409,0.000268,NaN,NaN
392,LDO,Defi,YieldDesk_Main,5,0.00025,0.0,3.32,0.000828,0.0,NaN,NaN
384,KIN,Other,Others - Asset,5,4539546619.0,NaN,0,0.0,NaN,NaN,NaN
28,ALICE,Exchange,FTX - CnC,4,60000.047441,-0.267206,NaN,NaN,NaN,NaN,NaN


In [45]:
# process the collateral table and insert it into the liquidity tier summary
# for COFA value of collateral, use default first, there is a "Collateral" column in COFA which will override the default
cofa_collateral_p1 = cofa_original[cofa_original["Collateral"].notnull()]
cofa_collateral_p2 = cofa_original[cofa_original["Collateral"].isnull()]
cofa_collateral_p1["Default"] = cofa_collateral_p1["Collateral"]
cofa_collateral = pd.concat([cofa_collateral_p1, cofa_collateral_p2])[["Coin", "Default"]]
cofa_collateral.columns = ["Coin", "COFA"]
collateral = collateral.merge(cofa_collateral, on = "Coin", how = "left")
collateral = collateral.merge(coin_price, on = "Coin", how = "left")
collateral["User Collateral USD Value"] = collateral["User Collateral"] * collateral["Price"]
collateral["Inst Collateral USD Value"] = collateral["Inst Collateral"] * collateral["Price"]


In [46]:
# get the whole apy-template table and insert it into the liquidity tier summary
apy_template = template.drop([0])
apy_template.fillna(value = "N/A", inplace = True)
#some people leave spaces in apy sheet, need to remove them
apy_template.replace([" ", "", "  "], "N/A", inplace = True)

In [47]:
#path = "C:/celsius/Liquidity/Liquidity_Tier_Summary.xlsx"
path = "excel_outputs/Liquidity_Tier_Summary.xlsx"
lp = openpyxl.load_workbook(path)
lp.remove(lp['Data'])
lp.remove(lp['Price'])
lp.remove(lp['Pivot_Data'])
lp.remove(lp['APY'])
lp.remove(lp['COFA'])
lp.remove(lp['Collateral'])
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = lp
coin_tiers[filter3].to_excel(writer, sheet_name = 'Data', index = False)
coin_price.to_excel(writer, sheet_name = 'Price', index = False)
df2[col_names].to_excel(writer, sheet_name = 'Pivot_Data', index = False)
apy_template.to_excel(writer, sheet_name = 'APY', index = False)
cofa_original.to_excel(writer, sheet_name = 'COFA', index = False)
collateral.to_excel(writer, sheet_name = 'Collateral', index = False)
writer.close()
lp.close()

## following code will push the data to the shared google sheet 

In [48]:
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from datetime import datetime as dt


In [49]:
scope = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive'
]
last_updated = "Updated at - " + dt.now().strftime("%m/%d/%Y, %H:%M:%S") +'IL'
# this is ID for testing
#SPREADSHEET_ID = '1IptNC0hEhwvuyfI4m2kP9-rR-3jDAOhgQaPNkW2I_QQ'
# this is ID for waterfall sheet
SPREADSHEET_ID = "1ZkSLZH2QwHnfdSpQUWAv2qum6xzhemngjWQBJBn_KeM"
#SPREADSHEET_ID = "1ULOqqkYP7DDhVkZBoEA3i-Ka4x_YBPtAPuIzb84Ekrg" # GAB LAST TEST
#SPREADSHEET_ID = "1maKKENmcDKNHWOVG2cJrcObSSL3GD8Fj4X1ZBuWiXOs" # GAB PORTFOLIO TEST
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('write_token.json'):
    creds = Credentials.from_authorized_user_file('write_token.json', scope)
'''flow = InstalledAppFlow.from_client_secrets_file('client_secret_510599516312-aj6d72c90n3fmrbf3ou6gromil06pr8c.apps.googleusercontent.com.json', scope)
creds = flow.run_local_server(port=0)
with open('write_token.json', 'w') as token:
    token.write(creds.to_json())'''
service = build('sheets', 'v4', credentials=creds)

In [50]:
response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Categories!A1",
    body=dict(
        majorDimension='ROWS',
        values=[[last_updated]]
)
).execute()

In [51]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Data",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Data!A1",
    body=dict(
        majorDimension='ROWS',
        values=coin_tiers[filter3].fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

In [52]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Price",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Price!A1",
    body=dict(
        majorDimension='ROWS',
        values=coin_price.fillna(value = 0).T.reset_index().T.values.tolist())
).execute()

In [53]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="APY",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="APY!A1",
    body=dict(
        majorDimension='ROWS',
        values=apy_template.fillna(value = 0).T.reset_index().T.values.tolist())
).execute()

In [54]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="COFA",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="COFA!A1",
    body=dict(
        majorDimension='ROWS',
        values=cofa_original.fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

In [55]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Collateral",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Collateral!A1",
    body=dict(
        majorDimension='ROWS',
        values=collateral.fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

In [56]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Coin_Total_Asset_Liability",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Coin_Total_Asset_Liability!A1",
    body=dict(
        majorDimension='ROWS',
        values=coin_asset_liability.fillna(value = 0).T.reset_index().T.values.tolist())
).execute()

In [57]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Pivot_Data",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Pivot_Data!A1",
    body=dict(
        majorDimension='ROWS',
        values=df2[col_names].fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

## below code is to cpoy this freeze waterfall into the archive

In [58]:
scope = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive',
'https://www.googleapis.com/auth/drive.file'
]

# this is ID for waterfall live, but we just updated it using freeze data
SPREADSHEET_ID = "1ZkSLZH2QwHnfdSpQUWAv2qum6xzhemngjWQBJBn_KeM"
#SPREADSHEET_ID = "1ULOqqkYP7DDhVkZBoEA3i-Ka4x_YBPtAPuIzb84Ekrg" # GAB LAST TEST
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('drive_file_token.json'):
        creds = Credentials.from_authorized_user_file('drive_file_token.json', scope)
'''flow = InstalledAppFlow.from_client_secrets_file('client_secret_510599516312-aj6d72c90n3fmrbf3ou6gromil06pr8c.apps.googleusercontent.com.json', scope)
creds = flow.run_local_server(port=0)
with open('drive_file_token.json', 'w') as token:
    token.write(creds.to_json())'''
service = build('drive', 'v3', credentials=creds)



In [59]:
# extract the most recent freeze id from the url provided and get the file name using that id
freeze = pd.read_excel("excel_input/freeze_address.xlsx")
freeze.columns = freeze.iloc[0]
freeze.drop([0], inplace = True)
freeze_address = str(freeze.iloc[-1]["Freeze URL"])
#print(freeze_address)
freeze_id = freeze_address.split("/")[5]

#get the file name of most recent freeze sheet, which include the date of the freeze
response = service.files().get(fileId=freeze_id).execute()
freeze_name  = "Portfolio Waterfall - " + response["name"]

In [60]:
freeze_name

'Portfolio Waterfall - Freeze- January 4th, 2022'

In [61]:
# after updating the portfolio waterall live, copied it to the archive folder
archive_folder_id = "13f2xspl16wzdHBRwhgnEkKj80zLL9Rzu"
newfile = {'name': freeze_name,  'parents' : [archive_folder_id]}
response = service.files().copy(fileId=SPREADSHEET_ID, body=newfile).execute()